In [16]:
! pip install pydot

In [13]:
! pip install textX[cli]

In [1]:
#To make a meta-model from a file
from textx import metamodel_from_file

In [2]:
#Make the meta-model
metamodel = metamodel_from_file('stepsEBNF.tx')

In [3]:
#Create the syntax tree
syntaxTree = metamodel.model_from_file('example.tx')

In [12]:
print(syntaxTree.objects)

[<textx:stepsEBNF.Object instance at 0x2e9a6738050>]


In [ ]:
#Como vaina le saco eso, porque solo me deja syntaxTree.objects (arriba)
print(syntaxTree.classDef)

AttributeError: 'ClassDiagram' object has no attribute 'classDef'

In [15]:
!textx generate stepsEBNF.tx --target dot

Generating dot target from models:
c:\Users\kracr\OneDrive\Desktop\Discretas3\Context Free Grammars\stepsEBNF.tx
-> c:\Users\kracr\OneDrive\Desktop\Discretas3\Context Free Grammars\stepsEBNF.dot
     To convert to png run "dot -Tpng -O stepsEBNF.dot"


In [ ]:
import pydot

(graph,) = pydot.graph_from_dot_file('stepsEBNF.dot')
graph.write_png('intento.png')

FileNotFoundError: [WinError 2] "dot" not found in path.